In [ ]:
%%python3 -m venv ../../myenv
%%source ../../myenv/bin/activate

In [ ]:
%pip install -r ../../requirements.txt

In [ ]:
from dotenv import load_dotenv
import pandas as pd
import os
import boto3

In [ ]:
data_folder = "../data"
test_file_path = os.path.join(data_folder, "merged_instructions.csv")
df = pd.read_csv(test_file_path)
df['instruction'].head()

In [ ]:
from botocore.exceptions import ClientError
import json
result_data = []

# Create a Bedrock Runtime client in the AWS Region of your choice.
client = boto3.client("bedrock-runtime", region_name="us-west-2")

# Set the model ID, e.g., Llama 3 70b Instruct.
model_id = "us.meta.llama3-2-11b-instruct-v1:0"

for index, row in df.iterrows():
    message = row['instruction']

    prompt = f"You are a helpful assistant that generates code snippets based on user instructions." \
                "The method signature should be according to the given in the example part.\n" \
                "Use the following guidlines in generation: \n" \
                " - Provide the code only\n" \
                " - Do not output any other texts or test cases results.\n" \
                " - Include the necessary import statements on the top even if they are usually not needed like - re, typing, itertools, Split etc. \n" \
                " - Do not include any comments in the code\n" \
                " - Do not forget to consider the edge cases\n" \
                " - Avoid recursion if possible\n" \
                " - Lastly, verify the code has no compilations error and contains the necessary imports.\n"
    

    # Embed the prompt in Llama 3's instruction format.
    formatted_prompt = f"""
    <|begin_of_text|><|start_header_id|>user<|end_header_id|>
    {prompt + message}
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """

    # Format the request payload using the model's native structure.
    native_request = {
        "prompt": formatted_prompt,
        "max_gen_len": 1024,
        "temperature": 0.5,
}

    # Convert the native request to JSON.
    request = json.dumps(native_request)

    try:
        # Invoke the model with the request.
        response = client.invoke_model(modelId=model_id, body=request)

    except (ClientError, Exception) as e:
        print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
        exit(1)

    # Decode the response body.
    model_response = json.loads(response["body"].read())

    # Extract and print the response text.
    output = model_response["generation"]

    result_data.append({
        "id": row['id'],
        "response": output
    })

    print("Completed:", row['id'])

result_df = pd.DataFrame(result_data)

In [ ]:
# Create a json from the result_df
result_json = result_df.to_json(orient="records", indent=4)
# save json in the data_folder os.path.join(data_folder, "submission.json")
with open("llama_3.2_11B_submission.json", "w") as json_file:
    json_file.write(result_json)